In [1]:
import os
import torch
import argparse
import itertools
import numpy as np
from tqdm import tqdm
import torch.optim as optim
from torchvision.utils import save_image
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import get_rank, init_process_group, destroy_process_group, all_gather, get_world_size
from torch import Tensor
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from glob import glob
from torch.utils.data.distributed import DistributedSampler
import random
from conditionDiffusion.unet import Unet
from conditionDiffusion.embedding import ConditionalEmbedding
from conditionDiffusion.utils import get_named_beta_schedule
from conditionDiffusion.diffusion import GaussianDiffusion
from conditionDiffusion.Scheduler import GradualWarmupScheduler
from PIL import Image
print(f"GPUs used:\t{torch.cuda.device_count()}")
device = torch.device("cuda",4)
print(f"Device:\t\t{device}")
import pytorch_model_summary as tms

GPUs used:	8
Device:		cuda:4


In [2]:
class_list=['유형1','유형2']
params={'image_size':1024,
        'lr':1e-5,
        'beta1':0.5,
        'beta2':0.999,
        'batch_size':4,
        'epochs':1000,
        'n_classes':None,
        'data_path':'../../data/origin_type/BRNT/',
        'image_count':5000,
        'inch':3,
        'modch':32,
        'outch':3,
        'chmul':[1,2,4,8,16,32,32],
        'numres':2,
        'dtype':torch.float32,
        'cdim':10,
        'useconv':False,
        'droprate':0.1,
        'T':1000,
        'w':1.8,
        'v':0.3,
        'multiplier':2.5,
        'threshold':0.1,
        'ddim':True,
        }


In [3]:
trans = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])

def transback(data:Tensor) -> Tensor:
    return data / 2 + 0.5

class CustomDataset(Dataset):
    """COCO Custom Dataset compatible with torch.utils.data.DataLoader."""
    def __init__(self,parmas, images,label):
        
        self.images = images
        self.args=parmas
        self.label=label
        
    def trans(self,image):
        if random.random() > 0.5:
            transform = transforms.RandomHorizontalFlip(1)
            image = transform(image)
            
        if random.random() > 0.5:
            transform = transforms.RandomVerticalFlip(1)
            image = transform(image)
            
        return image
    
    def __getitem__(self, index):
        image=self.images[index]
        label=self.label[index]
        image = self.trans(image)
        return image,label
    
    def __len__(self):
        return len(self.images)


image_label=[]
image_path=[]
for i in tqdm(range(len(class_list))):
    image_list=glob(params['data_path']+class_list[i]+'/*.jpeg')
    for j in range(len(image_list)):
        image_path.append(image_list[j])
        image_label.append(i)
        
train_images=torch.zeros((len(image_path),params['inch'],params['image_size'],params['image_size']))
for i in tqdm(range(len(image_path))):
    train_images[i]=trans(Image.open(image_path[i]).convert('RGB').resize((params['image_size'],params['image_size'])))
train_dataset=CustomDataset(params,train_images,image_label)
dataloader=DataLoader(train_dataset,batch_size=params['batch_size'],shuffle=True)

 15%|█▍        | 551/3722 [01:48<05:06, 10.34it/s]

In [ ]:
net = Unet(in_ch = params['inch'],
            mod_ch = params['modch'],
            out_ch = params['outch'],
            ch_mul = params['chmul'],
            num_res_blocks = params['numres'],
            cdim = params['cdim'],
            use_conv = params['useconv'],
            droprate = params['droprate'],
            dtype = params['dtype']
            ).to(device)
cemblayer = ConditionalEmbedding(len(class_list), params['cdim'], params['cdim']).to(device)
betas = get_named_beta_schedule(num_diffusion_timesteps = params['T'])
diffusion = GaussianDiffusion(
                    dtype = params['dtype'],
                    model = net,
                    betas = betas,
                    w = params['w'],
                    v = params['v'],
                    device = device
                )
optimizer = torch.optim.AdamW(
                itertools.chain(
                    diffusion.model.parameters(),
                    cemblayer.parameters()
                ),
                lr = params['lr'],
                weight_decay = 1e-4
            )

cosineScheduler = optim.lr_scheduler.CosineAnnealingLR(
                            optimizer = optimizer,
                            T_max = params['epochs']/100,
                            eta_min = 0,
                            last_epoch = -1
                        )
warmUpScheduler = GradualWarmupScheduler(
                        optimizer = optimizer,
                        multiplier = params['multiplier'],
                        warm_epoch = params['epochs'] // 10,
                        after_scheduler = cosineScheduler,
                        last_epoch = 0
                    )
checkpoint=torch.load(f'../../model/conditionDiff/details/BRNT/ckpt_111_checkpoint.pt')
diffusion.model.load_state_dict(checkpoint['net'])
cemblayer.load_state_dict(checkpoint['cemblayer'])
optimizer.load_state_dict(checkpoint['optimizer'])
warmUpScheduler.load_state_dict(checkpoint['scheduler'])



AttributeError: 'dict' object has no attribute 'load_state_dict'

In [ ]:
for epc in range(110,params['epochs']):
    diffusion.model.train()
    cemblayer.train()
    total_loss=0
    steps=0
    with tqdm(dataloader, dynamic_ncols=True) as tqdmDataLoader:
        for img, lab in tqdmDataLoader:
            b = img.shape[0]
            optimizer.zero_grad()
            x_0 = img.to(device)
            lab = lab.to(device)
            cemb = cemblayer(lab)
            cemb[np.where(np.random.rand(b)<params['threshold'])] = 0
            loss = diffusion.trainloss(x_0, cemb = cemb)
            loss.backward()
            optimizer.step()
            steps+=1
            total_loss+=loss.item()
            tqdmDataLoader.set_postfix(
                ordered_dict={
                    "epoch": epc + 1,
                    "loss: ": total_loss/steps,
                    "batch per device: ":x_0.shape[0],
                    "img shape: ": x_0.shape[1:],
                    "LR": optimizer.state_dict()['param_groups'][0]["lr"]
                }
            )
    warmUpScheduler.step()
    if (epc) % 10 == 0:
        diffusion.model.eval()
        cemblayer.eval()
        # generating samples
        # The model generate 80 pictures(8 per row) each time
        # pictures of same row belong to the same class
        all_samples = []
        each_device_batch =10
        with torch.no_grad():
            lab = torch.ones(len(class_list), each_device_batch // len(class_list)).type(torch.long) \
            * torch.arange(start = 0, end = len(class_list)).reshape(-1, 1)
            lab = lab.reshape(-1, 1).squeeze()
            lab = lab.to(device)
            cemb = cemblayer(lab)
            genshape = (each_device_batch , 3, params['image_size'], params['image_size'])
            if params['ddim']:
                generated = diffusion.ddim_sample(genshape, 50, 0, 'linear', cemb = cemb)
            else:
                generated = diffusion.sample(genshape, cemb = cemb)
            img = transback(generated)
            img = img.reshape(len(class_list), each_device_batch // len(class_list), 3, params['image_size'], params['image_size']).contiguous()
            all_samples.append(img)
            samples = torch.concat(all_samples, dim = 1).reshape(each_device_batch, 3,params['image_size'], params['image_size'])

        save_image(samples,f'../../result/Detail/BRNT/generated_{epc+1}_pict.png', nrow = each_device_batch // len(class_list))
        # save checkpoints
        checkpoint = {
                            'net':diffusion.model.state_dict(),
                            'cemblayer':cemblayer.state_dict(),
                            'optimizer':optimizer.state_dict(),
                            'scheduler':warmUpScheduler.state_dict()
                        }
        torch.save(checkpoint, f'../../model/conditionDiff/details/BRNT/ckpt_{epc+1}_checkpoint.pt')
    torch.cuda.empty_cache()
    

100%|██████████| 931/931 [24:30<00:00,  1.58s/it, epoch=11, loss: =0.419, batch per device: =2, img shape: =torch.Size([3, 1024, 1024]), LR=1e-5]


Start generating(ddim)...


100%|██████████| 50/50 [01:32<00:00,  1.86s/it]


ending sampling process(ddim)...


100%|██████████| 931/931 [25:00<00:00,  1.61s/it, epoch=21, loss: =0.0392, batch per device: =2, img shape: =torch.Size([3, 1024, 1024]), LR=1.15e-5]


...

ending sampling process(ddim)...


100%|██████████| 931/931 [24:37<00:00,  1.59s/it, epoch=101, loss: =0.0232, batch per device: =2, img shape: =torch.Size([3, 1024, 1024]), LR=2.35e-5]


Start generating(ddim)...


100%|██████████| 50/50 [01:48<00:00,  2.17s/it]


ending sampling process(ddim)...
